<a href="https://colab.research.google.com/github/Harsh0487/Harsh_INFO5731_fall2024/blob/main/INFO5731_harsh_patel_Assigment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:


!pip install requests beautifulsoup4 pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_imdb_reviews(movie_id, max_reviews=1000):
    reviews = []
    page_key = None
    base_url = f'https://www.imdb.com/title/{movie_id}/reviews/_ajax'

    while len(reviews) < max_reviews:
        url = base_url if page_key is None else f'{base_url}?paginationKey={page_key}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        review_containers = soup.find_all('div', class_='review-container')

        if not review_containers:
            print("No more reviews found.")
            break

        for container in review_containers:
            if len(reviews) >= max_reviews:
                break

            try:
                review_title = container.find('a', class_='title').get_text(strip=True)
                review_content = container.find('div', class_='text').get_text(strip=True)
                review_rating = container.find('span', class_='rating-other-user-rating')
                review_rating = review_rating.get_text(strip=True) if review_rating else None
                review_date = container.find('span', class_='review-date').get_text(strip=True)
                review_username = container.find('span', class_='display-name-link').get_text(strip=True)

                reviews.append({
                    'Title': review_title,
                    'Content': review_content,
                    'Rating': review_rating,
                    'Date': review_date,
                    'Username': review_username
                })
            except AttributeError:
                continue

        pagination_key_tag = soup.find('div', class_='load-more-data')
        page_key = pagination_key_tag.get('data-key') if pagination_key_tag else None

        if not page_key:
            break

        time.sleep(1)

    return reviews

def save_reviews_to_csv(movie_ids, filename="imdb_reviews.csv"):
    all_reviews = []

    for movie_id in movie_ids:
        print(f"Scraping reviews for movie ID: {movie_id}")
        reviews = get_imdb_reviews(movie_id)
        all_reviews.extend(reviews)

        if len(all_reviews) >= 1000:
            break

    df = pd.DataFrame(all_reviews[:1000])
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} reviews to {filename}")

if __name__ == "__main__":
    movie_ids = ['tt14826022', 'tt1630029']
    save_reviews_to_csv(movie_ids)




Scraping reviews for movie ID: tt14826022
Scraping reviews for movie ID: tt1630029
Saved 1000 reviews to imdb_reviews.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:

!pip install nltk pandas

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv('imdb_reviews.csv')

print("Original Data:")
print(df[['Title', 'Content']].head())

def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

df['Cleaned_Content'] = df['Content'].apply(remove_special_characters)
print("\nAfter Removing Special Characters:")
print(df[['Content', 'Cleaned_Content']].head())

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

df['Cleaned_Content'] = df['Cleaned_Content'].apply(remove_numbers)
print("\nAfter Removing Numbers:")
print(df[['Content', 'Cleaned_Content']].head())

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in stop_words])

df['Cleaned_Content'] = df['Cleaned_Content'].apply(remove_stopwords)
print("\nAfter Removing Stopwords:")
print(df[['Content', 'Cleaned_Content']].head())

df['Cleaned_Content'] = df['Cleaned_Content'].apply(lambda x: x.lower())
print("\nAfter Lowercasing Texts:")
print(df[['Content', 'Cleaned_Content']].head())

stemmer = PorterStemmer()

def apply_stemming(text):
    words = text.split()
    return ' '.join([stemmer.stem(word) for word in words])

df['Stemmed_Content'] = df['Cleaned_Content'].apply(apply_stemming)
print("\nAfter Stemming:")
print(df[['Cleaned_Content', 'Stemmed_Content']].head())

lemmatizer = WordNetLemmatizer()

def apply_lemmatization(text):
    words = text.split()
    return ' '.join([lemmatizer.lemmatize(word) for word in words])

df['Lemmatized_Content'] = df['Cleaned_Content'].apply(apply_lemmatization)
print("\nAfter Lemmatization:")
print(df[['Stemmed_Content', 'Lemmatized_Content']].head())

df.to_csv('cleaned_imdb_reviews.csv', index=False)
print("Cleaned data saved to cleaned_imdb_reviews.csv")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Original Data:
                                               Title  \
0            The racism actually got on my nerves...   
1                                      Those People.   
2                             Astonishingly annoying   
3                   You People made a terrible movie   
4  The most annoying film of 2023, it made me so ...   

                                             Content  
0  The black side actually aggravated me more tha...  
1  You know, the ones who wrote all the cliched n...  
2  It's been some time that I've been so bewilder...  
3  How did this ever get made? Old Man Jonah Hill...  
4  When mum turned to watch this godforsaken movi...  

After Removing Special Characters:
                                             Content  \
0  The black side actually aggravated me more tha...   
1  You know, the ones who wrote all the cliched n...   
2  It's been some time that I've been so bewilder...   
3  How did this ever get made? Old Man Jonah Hill...   
4 

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:

!pip install nltk

import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize, ne_chunk
from nltk.tree import Tree
from collections import Counter

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

df = pd.read_csv('cleaned_imdb_reviews.csv')

def pos_analysis(text):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

    for word, tag in pos_tags:
        if tag.startswith('NN'):
            counts['Noun'] += 1
        elif tag.startswith('VB'):
            counts['Verb'] += 1
        elif tag.startswith('JJ'):
            counts['Adjective'] += 1
        elif tag.startswith('RB'):
            counts['Adverb'] += 1

    return counts

total_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

for content in df['Lemmatized_Content']:
    counts = pos_analysis(content)
    for key in total_counts:
        total_counts[key] += counts[key]

print("Total Parts of Speech Counts:")
print(total_counts)

def constituency_parsing(text):
    sentences = nltk.sent_tokenize(text)
    for sent in sentences:
        tokens = nltk.word_tokenize(sent)
        tagged = nltk.pos_tag(tokens)
        tree = ne_chunk(tagged)
        print("\nConstituency Parsing Tree:")
        tree.pretty_print()

example_text = df['Lemmatized_Content'].iloc[0]
constituency_parsing(example_text)

def named_entity_recognition(text):
    sentences = nltk.sent_tokenize(text)
    entities = []
    for sent in sentences:
        tokens = nltk.word_tokenize(sent)
        tagged = nltk.pos_tag(tokens)
        tree = ne_chunk(tagged)
        for subtree in tree.subtrees():
            if isinstance(subtree, Tree):
                if subtree.label() == 'PERSON':
                    person = ' '.join([token for token, pos in subtree.leaves()])
                    entities.append(('PERSON', person))
                elif subtree.label() == 'ORGANIZATION':
                    org = ' '.join([token for token, pos in subtree.leaves()])
                    entities.append(('ORGANIZATION', org))
                elif subtree.label() == 'GPE':
                    gpe = ' '.join([token for token, pos in subtree.leaves()])
                    entities.append(('GPE', gpe))

    entity_counts = Counter(entities)
    return entity_counts

total_entities = {'PERSON': 0, 'ORGANIZATION': 0, 'GPE': 0}

for content in df['Lemmatized_Content']:
    counts = named_entity_recognition(content)
    for entity_type, count in counts.items():
        total_entities[entity_type[0]] += count

print("\nTotal Named Entity Counts:")
print(total_entities)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Total Parts of Speech Counts:
{'Noun': 43319, 'Verb': 19189, 'Adjective': 22073, 'Adverb': 9421}

Constituency Parsing Tree:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      S                                                                                                                                                                                                                                                                                                                    

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [ ]:
https://drive.google.com/file/d/1lg2bM0ClsRuH65G_YR_eda0h6f1w5xOX/view?usp=sharing

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:


# Challenges Faced:-
# 1. Data Cleaning Complexity:- Ensuring that the data cleaning process was thorough and effective was challenging. Each step, from removing special characters to lemmatization, required careful consideration to ensure that important information was not lost while also making the text suitable for analysis.

# 2.Understanding NLP Libraries:- Transitioning from using spaCy to NLTK required a deeper understanding of the functionalities and limitations of NLTK. Adapting to the differences in approaches to tasks like Named Entity Recognition (NER) and parsing took some time.

# 3.Debugging Issues:- Encountering LookupError due to missing NLTK resources was initially frustrating. It highlighted the importance of ensuring that all necessary data is downloaded before running NLP tasks.



# Enjoyable Aspects:-

# 1.Hands-On Learning:- Applying natural language processing techniques to real-world data was a rewarding experience. It was satisfying to see how various cleaning techniques could significantly improve the quality of the text data.

# 2.Exploring NLP Techniques:- The assignment provided an opportunity to explore different NLP techniques such as Parts of Speech (POS) tagging, constituency parsing, and Named Entity Recognition. Understanding how these techniques work and their applications in text analysis was particularly fascinating.

# 3.Data Analysis:- Analyzing the cleaned data for insights and patterns was engaging. The potential to derive meaningful conclusions from user reviews added a layer of excitement to working with text data.


# Opinion on Provided Time:-
# The time allocated for this assignment seemed reasonable for someone with a basic understanding of Python and natural language processing concepts. However, for those who may be new to NLP or programming in general, additional time might be beneficial to grasp the intricacies involved in text processing and analysis.